In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-b1700331af6d3576.parquet', 'test': 'data/test-00000-of-00001-460abe60f17dbc1c.parquet'}
df = pd.read_parquet("hf://datasets/ShashiVish/cover-letter-dataset/" + splits["train"])
print(df.columns)
print(df.iloc[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Index(['Job Title', 'Preferred Qualifications', 'Hiring Company',
       'Applicant Name', 'Past Working Experience',
       'Current Working Experience', 'Skillsets', 'Qualifications',
       'Cover Letter'],
      dtype='object')
Job Title                                                 Senior Java Developer
Preferred Qualifications             5+ years of experience in Java Development
Hiring Company                                                           Google
Applicant Name                                                         John Doe
Past Working Experience                       Java Developer at XYZ for 3 years
Current Working Experience             Senior Java Developer at ABC for 2 years
Skillsets                                     Java, Spring Boot, Hibernate, SQL
Qualifications                                          BSc in Computer Science
Cover Letter                  I am writing to express my interest in the Sen...
Name: 0, dtype: object


In [ ]:
def format_input(example):
    return f"""Write a personalized and professional cover letter based on the following details:

Applicant Name: {example['Applicant Name']}
Job Title: {example['Job Title']}
Company: {example['Hiring Company']}
Skillsets: {example['Skillsets']}
Qualifications: {example['Qualifications']}
Preferred Qualifications: {example['Preferred Qualifications']}
Past Working Experience: {example['Past Working Experience']}
Current Working Experience: {example['Current Working Experience']}
"""

df["input"] = df.apply(format_input, axis=1)
df["output"] = df["Cover Letter"]
from datasets import Dataset
train_dataset = Dataset.from_pandas(df[["input", "output"]])

In [ ]:
from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

def preprocess(example):
    model_inputs = tokenizer(example["input"], max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(example["output"], max_length=512, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = train_dataset.map(preprocess)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/813 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

#model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-coverletter",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"  # <--- Disable W&B
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)



config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<ipython-input-5-3679007647>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,22.358800
20,11.257400
30,4.175500
40,3.518400
50,2.849900
60,2.260600
70,1.749800
80,1.315600
90,1.074400
100,0.847800


TrainOutput(global_step=306, training_loss=1.9995156339570588, metrics={'train_runtime': 1070.2795, 'train_samples_per_second': 2.279, 'train_steps_per_second': 0.286, 'total_flos': 1670123182620672.0, 'train_loss': 1.9995156339570588, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./finetuned-flan-t5-coverletter")
tokenizer.save_pretrained("./finetuned-flan-t5-coverletter")

('./finetuned-flan-t5-coverletter/tokenizer_config.json',
 './finetuned-flan-t5-coverletter/special_tokens_map.json',
 './finetuned-flan-t5-coverletter/spiece.model',
 './finetuned-flan-t5-coverletter/added_tokens.json',
 './finetuned-flan-t5-coverletter/tokenizer.json')

In [ ]:
import torch

def generate_cover_letter(
    name, company, job_position,
    skillsets, qualifications,
    preferred_qualifications,
    past_experience, current_experience
):
    device = model.device  # ensure consistent device usage

    prompt = f"""Write a personalized and professional cover letter based on the following details:

Applicant Name: {name}
Job Title: {job_position}
Company: {company}
Skillsets: {skillsets}
Qualifications: {qualifications}
Preferred Qualifications: {preferred_qualifications}
Past Working Experience: {past_experience}
Current Working Experience: {current_experience}

The letter should be:
- Formal but engaging
- Clearly structured (greeting, introduction, body, closing)
- Aligned with the role and company
- Focused on strengths and fit

Generate a complete, well-formatted cover letter.
"""

    # Move input to model's device
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=768,
            temperature=0.8,
            num_return_sequences=1,
            do_sample=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
!ls ./finetuned-flan-t5-coverletter

config.json		special_tokens_map.json  tokenizer.json
generation_config.json	spiece.model
model.safetensors	tokenizer_config.json


In [ ]:
!zip -r flan_t5_coverletter.zip ./finetuned-flan-t5-coverletter
from google.colab import files
files.download("flan_t5_coverletter.zip")

  adding: finetuned-flan-t5-coverletter/ (stored 0%)
  adding: finetuned-flan-t5-coverletter/tokenizer.json (deflated 74%)
  adding: finetuned-flan-t5-coverletter/generation_config.json (deflated 29%)
  adding: finetuned-flan-t5-coverletter/tokenizer_config.json (deflated 95%)
  adding: finetuned-flan-t5-coverletter/spiece.model (deflated 48%)
  adding: finetuned-flan-t5-coverletter/special_tokens_map.json (deflated 85%)
  adding: finetuned-flan-t5-coverletter/model.safetensors (deflated 7%)
  adding: finetuned-flan-t5-coverletter/config.json (deflated 62%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# User-provided details
name = "Diego Lopez"
company = "OpenAI"
job_position = "Machine Learning Engineer"
skillsets = "Python, Deep Learning, NLP"
qualifications = "MSc in Data Science"
preferred_qualifications = "PhD preferred, but MSc accepted"
past_experience = "3 years at IBM as a Data Scientist"
current_experience = "Currently working at Google Research on language models"

cover_letter = generate_cover_letter(
    name, company, job_position,
    skillsets, qualifications,
    preferred_qualifications,
    past_experience, current_experience
)

print("=" * 50)
print("GENERATED COVER LETTER")
print("=" * 50)
print(cover_letter)

GENERATED COVER LETTER
I am writing to express my interest in the Machine Learning Engineer position at OpenAI. I am confident that I can provide this exceptional opportunity to take your skills and expertise to a new level. My attention to detail and proven ability to communicate a strong knowledge of Python and deep learning make me a strong candidate for the role. I am confident that my skills and experience will make me a great addition to the team. Thank you for considering my application. I look forward to the opportunity to discuss how my skills and experience make me a strong fit for the role. Sincerely, Diego Lopez
